## RAG

R -> Retrieval

A -> Augented

G - > Generation

* First the document is stored in vector database
* We take prompt from the user.
* Relevent information is **retrived** froom the Vector DB on the basis the prompt.
* The prompt and the context from the retrievd document is **Augmented**.
* This augmented prompt is used to **Generate** the response from LLM.

In [ ]:
# Requirements
!pip install faiss-cpu 
! pip install langchain-community
!pip install sentence-transformers

Access is denied.
Access is denied.
Access is denied.


In [6]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   --- ------------------------------------ 1.6/18.9 MB 14.9 MB/s eta 0:00:02
   ------------ --------------------------- 5.8/18.9 MB 17.6 MB/s eta 0:00:01
   -------------------------- ------------- 12.3/18.9 MB 22.9 MB/s eta 0:00:01
   ------------------------------------ --- 17.0/18.9 MB 23.0 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 21.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install langchain-community

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached marshmallow-3.26.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------------------ --------------- 1.6/2.5 MB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 12.2 MB/s eta 0:00:00

   ---------- ----------------------------- 2/8 [requests]
   -

In [8]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   -------------- ------------------------- 4.5/12.0 MB 22.2 MB/s eta 0:00:01
   --------------------------- ------------ 8.4/12.0 MB 20.5 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 20.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   --------------------------------------- 566.1/566.1 kB 19.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   - -------------------------------------- 4.5/113.8 MB 20.9 MB/s eta 0:00:06
   -- ------------------------------------- 7.3/113.8 MB 17.7 MB/s eta 0:00:06
   --- ------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [33]:
# Import Libraries
import os
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceEmbeddings
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import faiss
from langchain_community.vectorstores import FAISS

In [59]:
# Step 1: Configure the Models
# First lets configure the models

# LLM model
gemini_key = os.getenv("TestProject2")
genai.configure(api_key=gemini_key)
model = genai.GenerativeModel('gemini-2.5-flash')

# Configure Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [25]:
# Step 2: Get the document and extract the text

pdf_file = PdfReader(r"D:\GenAI Applications\Day4\sample.pdf")

raw_text = ''
for page in pdf_file.pages:
    text = page.extract_text()
    if text:
        raw_text = raw_text + text + '\n'

In [26]:
print(raw_text)

1  
MAJOR PROJECT REPORT 
at 
Sathyabama Institute of Science and Technology 
(Deemed to be University) 
 
Submitted in partial fulfillment of the requirements for the award of 
Bachelor of Engineering Degree in Computer Science and Engineering 
 
By 
Busupalli Harinath Reddy(Reg.No.38110063) 
Avala Pavan Kumar (Reg. No.38110058) 
 
 
 
 
 
 
 
 
 
DEPARTMENT OF COMPUTER SCIENCE AND ENGINEERING 
SCHOOL OF COMPUTING 
SATHYABAMA INSTITUTE OF SCIENCE AND TECHNOLOGY 
JEPPIAAR NAGAR, RAJIV GANDHI SALAI, 
CHENNAI – 600119, TAMILNADU 
 
 
MARCH 2022 

2  
 
SATHYABAMA 
INSTITUTE OF SCIENCE AND TECHNOLOGY 
(DEEMED TO BE UNIVERSITY) 
Accredited with Grade “A” by NAAC 
(Established under Section 3 of UGC Act, 1956) 
JEPPIAAR NAGAR, RAJIV GANDHI SALAI, CHENNAI– 600119 
www.sathyabamauniversity.ac.in 
 
 
DEPARTMENT OF COMPUTER SCIENCE AND ENGINEERING 
 
 
                                               BONAFIDE CERTIFICATE 
 
 
This is to certify that this Project Report is the bonafide work of Av

In [28]:
# Step 3: Chunking 
# First we need to split the text

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_text(raw_text)

In [29]:
len(raw_text)

37306

In [30]:
len(chunks)

49

In [34]:
# Create the Vector DB
vectorstore = FAISS.from_texts(chunks, embedding_model)

In [52]:
# Step 5: Get the prompt from the user
prompt = 'Give the brief introduction of the author of this report.'

In [53]:
# Step 6: Retrival (R)
retriver = vectorstore.as_retriever(search_kwargs={"k":3})
retrived_docs = retriver.invoke(prompt)

In [54]:
retrived_docs

[Document(id='e382ffe5-97b5-4786-882b-dfccbdcf73cf', metadata={}, page_content='Dr.S.Vigneshwari M.E., Ph.D., and Dr.L.Lakshmanan M.E., Ph.D., Heads of the  \nDepartment of Computer Science and Engineering for providing me necessary \nsupport and details at the right time during the progressive reviews. \nI would like to express my sincere and deep sense of gratitude to my Project Guide  Dr. \nR. AROUL CANESSANE  M.E., Ph.D.,  for her valuable guidance, suggestions and  \nconstant encouragement paved way for the successful completion of my project work. \nI wish to express my thanks to all Teaching and Non -teaching staff members of the  \nDepartment of Computer Science and Engineering who were helpful in many \nways for the completion of the project.\n5  \n \n                                 TABLE OF CONTENT \n \nINDEX \nNO \n                                             TITLE PAGE \nNO \n1.                                 ABSTRACT 6 \n2. INTRODUCTION 7 \n3.                            

In [55]:
context = '\n'.join([d.page_content for d in retrived_docs])

In [56]:
print(context)

Dr.S.Vigneshwari M.E., Ph.D., and Dr.L.Lakshmanan M.E., Ph.D., Heads of the  
Department of Computer Science and Engineering for providing me necessary 
support and details at the right time during the progressive reviews. 
I would like to express my sincere and deep sense of gratitude to my Project Guide  Dr. 
R. AROUL CANESSANE  M.E., Ph.D.,  for her valuable guidance, suggestions and  
constant encouragement paved way for the successful completion of my project work. 
I wish to express my thanks to all Teaching and Non -teaching staff members of the  
Department of Computer Science and Engineering who were helpful in many 
ways for the completion of the project.
5  
 
                                 TABLE OF CONTENT 
 
INDEX 
NO 
                                             TITLE PAGE 
NO 
1.                                 ABSTRACT 6 
2. INTRODUCTION 7 
3.                                     AIM 13 
4.                                   SCOPE 13
2  
 
SATHYABAMA 
INSTITUTE OF SCIEN

In [57]:
# Step 7: Augmenting(A)
augmented_prompt = '''
<ROLE> You are a helpful assistant using RAG.
<GOAL> Answer the question asked by the user. Here is the question {prompt}
<CONTEXT> Here are the documents retrieved from the vector database to support the answer which you have to generate{context}.

'''

In [60]:
# Step 8: Generation (G)
response = model.generate_content(augmented_prompt)
print(response.text)

SQL Server 2022 introduces a range of new features and improvements, primarily focusing on deep integration with Azure services, enhanced security, and performance optimizations. Here's a breakdown:

**1. Deep Integration with Azure:**

*   **Hybrid Data Platform:** It is the "most Azure-enabled release" of SQL Server, designed to provide seamless integration between on-premises and cloud environments.
*   **Azure Synapse Analytics Link:** Enables near real-time analytics by automatically replicating data from SQL Server to Azure Synapse Analytics, facilitating data warehousing and business intelligence without impacting operational workloads. It offloads ETL processes and minimizes the need for complex data movement.
*   **Microsoft Purview Integration:** Offers unified data governance, allowing for data discovery, classification, and lineage tracking across SQL Server and other data sources from the Azure Purview portal.
*   **Azure Arc-enabled SQL Server:** Extends Azure services an